In [1]:
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import openrouteservice as ors

In [8]:
# First define the map centered around Beira
m = folium.Map(location=[4.86, 100], tiles='cartodbpositron', zoom_start=8)

# Next load the delivery locations from CSV file at ../resources/data/idai_health_sites.csv
# ID, Lat, Lon, Open_From, Open_To, Needed_Amount
deliveries_data = pd.read_csv(
    'locations_fake.csv',
    index_col="ID",
    parse_dates=["Open_From", "Open_To"]
)

# Plot the locations on the map with more info in the ToolTip
for location in deliveries_data.itertuples():
    tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
        location.Index, location.Needed_Amount
    ))

    folium.Marker(
        location=[location.Lat, location.Lon],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(location.Index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)

# The vehicles are all located at the port of Beira
depot = [4.85, 100.74]

folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="bus", prefix='fa'),
    setZIndexOffset=1000
).add_to(m)

m

In [9]:
vehicles = list()
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx,
            start=list(reversed(depot)),
            # end=list(reversed(depot)),
            capacity=[300],
            time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )

# Next define the delivery stations
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
deliveries = list()
for delivery in deliveries_data.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.Lon, delivery.Lat],
            service=1200,  # Assume 20 minutes at each site
            amount=[delivery.Needed_Amount],
            time_windows=[[
                int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                int(delivery.Open_To.timestamp())
            ]]
        )
    )

In [10]:
api_key = '5b3ce3597851110001cf62482c3ae5d0a98f4a069134686d85176502'

In [11]:
ors_client = ors.Client(key=api_key)  # Get an API key from https://openrouteservice.org/dev/#/signup
result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)

# Add the output to the map
for color, route in zip(['green', 'red', 'blue'], result['routes']):
    decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                         }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m)

folium.LayerControl().add_to(m)
m

In [12]:
# Only extract relevant fields from the response
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
vehicles_df

,distance,amount,duration
vehicle,,,
0,12705,[293],1402
1,1651,[50],318
2,26118,[251],2047


In [13]:
# Create a list to display the schedule for all vehicles
stations = list()
for route in result['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # Station ID
                step["arrival"],  # Arrival time
                step["arrival"] + step.get("service", 0),  # Departure time

            ]
        )
    stations.append(vehicle)

In [14]:
df_stations_0 = pd.DataFrame(stations[0], columns=["Station ID", "Arrival", "Departure"])
df_stations_0['Arrival'] = pd.to_datetime(df_stations_0['Arrival'], unit='s')
df_stations_0['Departure'] = pd.to_datetime(df_stations_0['Departure'], unit='s')
df_stations_0

,Station ID,Arrival,Departure
0,Depot,2019-03-22 09:15:17,2019-03-22 09:15:17
1,5,2019-03-22 09:16:25,2019-03-22 09:36:25
2,1,2019-03-22 09:37:47,2019-03-22 09:57:47
3,7,2019-03-22 10:00:00,2019-03-22 10:20:00
4,11,2019-03-22 10:26:28,2019-03-22 10:46:28
5,10,2019-03-22 10:49:20,2019-03-22 11:09:20
6,12,2019-03-22 11:18:39,2019-03-22 11:38:39
7,Depot,2019-03-22 11:38:39,2019-03-22 11:38:39


In [15]:
df_stations_1 = pd.DataFrame(stations[1], columns=["Station ID", "Arrival", "Departure"])
df_stations_1['Arrival'] = pd.to_datetime(df_stations_1['Arrival'], unit='s')
df_stations_1['Departure'] = pd.to_datetime(df_stations_1['Departure'], unit='s')
df_stations_1

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:27:31,2019-03-22 08:27:31
1,13,2019-03-22 08:30:00,2019-03-22 08:50:00
2,8,2019-03-22 08:52:49,2019-03-22 09:12:49
3,Depot,2019-03-22 09:12:49,2019-03-22 09:12:49


In [16]:
df_stations_2 = pd.DataFrame(stations[2], columns=["Station ID", "Arrival", "Departure"])
df_stations_2['Arrival'] = pd.to_datetime(df_stations_2['Arrival'], unit='s')
df_stations_2['Departure'] = pd.to_datetime(df_stations_2['Departure'], unit='s')
df_stations_2

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:00:45,2019-03-22 08:00:45
1,3,2019-03-22 08:04:23,2019-03-22 08:24:23
2,2,2019-03-22 08:30:00,2019-03-22 08:50:00
3,9,2019-03-22 08:55:05,2019-03-22 09:15:05
4,6,2019-03-22 09:21:34,2019-03-22 09:41:34
5,4,2019-03-22 09:54:52,2019-03-22 10:14:52
6,Depot,2019-03-22 10:14:52,2019-03-22 10:14:52
